In [1]:
!pip install pinecone
!pip install pymongo
!pip install transformers
!pip install sentence_transformers
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.6/587.6 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.3/259.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.2 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 25.0
    Uninstalling packaging-25.0:
      Successfully uninstalled packaging-25.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 453.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 444.0/444.0 kB 21.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.74
    Uninstalling langchain-core-0.3.74:
      Successfully uninstalled langchain-core-0.3.74
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
g

In [2]:
from google.colab import userdata

PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')
MONGO_URI = userdata.get('MONGO_URI')
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

## Data Cleaning

In [3]:
from datasets import load_dataset

In [4]:
dataset=load_dataset("MongoDB/embedded_movies")

README.md: 0.00B [00:00, ?B/s]

sample_mflix.embedded_movies.json:   0%|          | 0.00/42.3M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1500 [00:00<?, ? examples/s]

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['plot', 'runtime', 'genres', 'fullplot', 'directors', 'writers', 'countries', 'poster', 'languages', 'cast', 'title', 'num_mflix_comments', 'rated', 'imdb', 'awards', 'type', 'metacritic', 'plot_embedding'],
        num_rows: 1500
    })
})

In [6]:
import pandas as pd
data=pd.DataFrame(dataset["train"])

In [7]:
data.shape

(1500, 18)

In [8]:
data.head()

,plot,runtime,genres,fullplot,directors,writers,countries,poster,languages,cast,title,num_mflix_comments,rated,imdb,awards,type,metacritic,plot_embedding
0,Young Pauline is left a lot of money when her ...,199.0,[Action],Young Pauline is left a lot of money when her ...,"[Louis J. Gasnier, Donald MacKenzie]","[Charles W. Goddard (screenplay), Basil Dickey...",[USA],https://m.media-amazon.com/images/M/MV5BMzgxOD...,[English],"[Pearl White, Crane Wilbur, Paul Panzer, Edwar...",The Perils of Pauline,0,None,"{'id': 4465, 'rating': 7.6, 'votes': 744}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.0007293965299999999, -0.026834568000000003,..."
1,A penniless young man tries to save an heiress...,22.0,"[Comedy, Short, Action]",As a penniless man worries about how he will m...,"[Alfred J. Goulding, Hal Roach]",[H.M. Walker (titles)],[USA],https://m.media-amazon.com/images/M/MV5BNzE1OW...,[English],"[Harold Lloyd, Mildred Davis, 'Snub' Pollard, ...",From Hand to Mouth,0,TV-G,"{'id': 10146, 'rating': 7.0, 'votes': 639}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.022837115, -0.022941574000000003, 0.014937..."
2,"Michael ""Beau"" Geste leaves England in disgrac...",101.0,"[Action, Adventure, Drama]","Michael ""Beau"" Geste leaves England in disgrac...",[Herbert Brenon],"[Herbert Brenon (adaptation), John Russell (ad...",[USA],None,[English],"[Ronald Colman, Neil Hamilton, Ralph Forbes, A...",Beau Geste,0,None,"{'id': 16634, 'rating': 6.9, 'votes': 222}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[0.00023330492999999998, -0.028511643000000003..."
3,"Seeking revenge, an athletic young man joins t...",88.0,"[Adventure, Action]",A nobleman vows to avenge the death of his fat...,[Albert Parker],"[Douglas Fairbanks (story), Jack Cunningham (a...",[USA],https://m.media-amazon.com/images/M/MV5BMzU0ND...,None,"[Billie Dove, Tempe Pigott, Donald Crisp, Sam ...",The Black Pirate,1,None,"{'id': 16654, 'rating': 7.2, 'votes': 1146}","{'nominations': 0, 'text': '1 win.', 'wins': 1}",movie,NaN,"[-0.005927917, -0.033394486, 0.0015323418, -0...."
4,An irresponsible young millionaire changes his...,58.0,"[Action, Comedy, Romance]","The Uptown Boy, J. Harold Manners (Lloyd) is a...",[Sam Taylor],"[Ted Wilde (story), John Grey (story), Clyde B...",[USA],https://m.media-amazon.com/images/M/MV5BMTcxMT...,[English],"[Harold Lloyd, Jobyna Ralston, Noah Young, Jim...",For Heaven's Sake,0,PASSED,"{'id': 16895, 'rating': 7.6, 'votes': 918}","{'nominations': 1, 'text': '1 nomination.', 'w...",movie,NaN,"[-0.0059373598, -0.026604708, -0.0070914757000..."


In [9]:
data=data.sample(80)

In [10]:
data.shape

(80, 18)

In [11]:
data.columns

Index(['plot', 'runtime', 'genres', 'fullplot', 'directors', 'writers',
       'countries', 'poster', 'languages', 'cast', 'title',
       'num_mflix_comments', 'rated', 'imdb', 'awards', 'type', 'metacritic',
       'plot_embedding'],
      dtype='object')

In [12]:
data.isnull().sum()

,0
plot,3
runtime,1
genres,0
fullplot,4
directors,0
writers,0
countries,0
poster,5
languages,0
cast,0


In [13]:
data=data.dropna(subset=["fullplot"])

In [14]:
data=data.drop(columns=["plot_embedding"])

In [15]:
data.isnull().sum()

,0
plot,0
runtime,1
genres,0
fullplot,0
directors,0
writers,0
countries,0
poster,4
languages,0
cast,0


## Data Ingestion

### MongoDB config

In [16]:
from pymongo.mongo_client import MongoClient

uri = MONGO_URI

# Create a new client and connect to the server
client = MongoClient(uri)

# Send a ping to confirm a successful connection
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [17]:
db=client["pinecone_mongo_db"]

In [18]:
collection=db["pinecone_mongo_db_collection"]

In [19]:
document=data.to_dict("records")

In [20]:
collection.insert_many(document)

InsertManyResult([ObjectId('68b80e677a853527a00ca716'), ObjectId('68b80e677a853527a00ca717'), ObjectId('68b80e677a853527a00ca718'), ObjectId('68b80e677a853527a00ca719'), ObjectId('68b80e677a853527a00ca71a'), ObjectId('68b80e677a853527a00ca71b'), ObjectId('68b80e677a853527a00ca71c'), ObjectId('68b80e677a853527a00ca71d'), ObjectId('68b80e677a853527a00ca71e'), ObjectId('68b80e677a853527a00ca71f'), ObjectId('68b80e677a853527a00ca720'), ObjectId('68b80e677a853527a00ca721'), ObjectId('68b80e677a853527a00ca722'), ObjectId('68b80e677a853527a00ca723'), ObjectId('68b80e677a853527a00ca724'), ObjectId('68b80e677a853527a00ca725'), ObjectId('68b80e677a853527a00ca726'), ObjectId('68b80e677a853527a00ca727'), ObjectId('68b80e677a853527a00ca728'), ObjectId('68b80e677a853527a00ca729'), ObjectId('68b80e677a853527a00ca72a'), ObjectId('68b80e677a853527a00ca72b'), ObjectId('68b80e677a853527a00ca72c'), ObjectId('68b80e677a853527a00ca72d'), ObjectId('68b80e677a853527a00ca72e'), ObjectId('68b80e677a853527a00ca7

### Pinecone Index Config

In [21]:
from pinecone import Pinecone

pc = Pinecone(api_key=PINECONE_API_KEY)
index = pc.Index("mongo-pinecone-rag")

In [22]:
index

In [23]:
from sentence_transformers import SentenceTransformer
embedding_model = SentenceTransformer("thenlper/gte-large")

modules.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/57.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/619 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/670M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [24]:
# One-time backfill script
docs = collection.find({})
for doc in docs:
    if 'fullplot' in doc:
        vector = embedding_model.encode(doc['fullplot']).tolist()
        index.upsert([(str(doc['_id']), vector)])
print("✅ Initial data uploaded to Pinecone")            # 3 min approx

✅ Initial data uploaded to Pinecone



 1️⃣ Initial Backfill (one-time)

When you first start, Pinecone is empty.

So you need to push the existing dataset from MongoDB → Pinecone.

This is usually a separate script you run once:


2️⃣ Realtime Sync (continuous)

After backfilling, run the change stream listener (the script I gave you earlier).

That script keeps watching MongoDB for new insert/update/delete operations and syncs them to Pinecone.

✅ So the workflow is:

1.Upload dataset to MongoDB Atlas.

2.Run the backfill script once → all data goes into Pinecone.

3.Start the realtime sync script → keeps Pinecone in sync forever.
```
# --- Change Stream Listener ---
cursor = collection.watch(full_document='updateLookup')
print("📡 Change stream is now open.")

while True:
    change = next(cursor)

    # Handle new inserts
    if change['operationType'] == 'insert':
        document = change['fullDocument']
        if 'fullplot' in document:
            vector = embedding_model.encode(document['fullplot']).tolist()
            upsert_data = (str(document['_id']), vector)
            index.upsert([upsert_data])
            print(f"✅ Inserted {document['_id']} into Pinecone")

    # Handle updates
    elif change['operationType'] == 'update':
        document = change['fullDocument']
        updated_fields = change['updateDescription']['updatedFields']

        if 'fullplot' in updated_fields:
            vector = embedding_model.encode(updated_fields['fullplot']).tolist()
            upsert_data = (str(document['_id']), vector)
            index.upsert([upsert_data])
            print(f"🔄 Updated {document['_id']} in Pinecone")

    # Handle deletes
    elif change['operationType'] == 'delete':
        index.delete(ids=[str(change['documentKey']['_id'])])
        print(f"❌ Deleted {change['documentKey']['_id']} from Pinecone")
```

## Retrival

In [25]:
def get_result(query,similar_result=3):
  embedding=embedding_model.encode(query)
  embedding=embedding.tolist()

  result=index.query(
    vector=embedding,
    top_k=similar_result,
  )
  return result

In [26]:
query="what is the best horror movie to watch and why?"

In [27]:
result=get_result(query)

In [28]:
result

{'matches': [{'id': '68b80e677a853527a00ca745',
              'score': 0.776882052,
              'values': []},
             {'id': '68b80e677a853527a00ca719',
              'score': 0.774049819,
              'values': []},
             {'id': '68b80e677a853527a00ca71c',
              'score': 0.76472193,
              'values': []}],
 'namespace': '',
 'usage': {'read_units': 1}}

## Generation

In [29]:
from bson.objectid import ObjectId

mylist=[]
for i in  range(len(result["matches"])):
  value=result["matches"][i]['id']
  mylist.append(collection.find_one({"_id": ObjectId(value)}))

print(mylist)

[{'_id': ObjectId('68b80e677a853527a00ca745'), 'plot': 'After rising from his volcanic grave, Godzilla is threatened by a mutated rosebush.', 'runtime': 104.0, 'genres': ['Action', 'Drama', 'Horror'], 'fullplot': 'A scientist combines the cells of a rose with those of Godzilla to create a biological creature more horrifying than any seen before. The two do battle after a destructive tour of Japan by Godzilla. A newly released version includes many new fight scenes that were cut out of the theatrical version.', 'directors': ['Kazuki Ohmori'], 'writers': ['Shinichirè Kobayashi (story)', 'Kazuki Ohmori'], 'countries': ['Japan'], 'poster': 'https://m.media-amazon.com/images/M/MV5BODZjOWU4YTgtNmUyNC00ZjQ2LTk3MzUtNmRlNzgzNzFkNDE4L2ltYWdlL2ltYWdlXkEyXkFqcGdeQXVyNTMxMjgxMzA@._V1_SY1000_SX677_AL_.jpg', 'languages': ['Japanese', 'English'], 'cast': ['Kunihiko Mitamura', 'Yoshiko Tanaka', 'Masanobu Takashima', 'Kèji Takahashi'], 'title': 'Godzilla vs. Biollante', 'num_mflix_comments': 0, 'rated':

In [30]:
combined_information = ""
for i in range(len(mylist)):
  fullplot=mylist[i]["fullplot"]
  title=mylist[i]["title"]
  combined_information += f"Title:{title}, fullplot: {fullplot}\n"

print(combined_information)

Title:Godzilla vs. Biollante, fullplot: A scientist combines the cells of a rose with those of Godzilla to create a biological creature more horrifying than any seen before. The two do battle after a destructive tour of Japan by Godzilla. A newly released version includes many new fight scenes that were cut out of the theatrical version.
Title:Search and Destroy, fullplot: A would-be movie producer strives to convince a self-help guru to let him film his book. As the guru is only interested in money, the producer solicits the help of a shady businessman and the guru's own secretary, who wants to write movie scripts (a horror story about brain-scraping monsters).
Title:Megiddo: The Omega Code 2, fullplot: Megiddo is a supernatural ride into a world teetering on the edge of the Apocalypse. It follows the rise of a Machiavellian leader bent on amassing the armies of the world for the battle of Armageddon while calamities of Biblical proportions pummel the Earth. Though both prequel and se

In [31]:
prompt = f"Query: {query}\nContinue to answer the query by using the fullplot only:\n{combined_information}."

In [32]:
from langchain_google_genai import ChatGoogleGenerativeAI

def load_model():
  llm = ChatGoogleGenerativeAI(model = "gemini-2.5-flash-lite",api_key = GOOGLE_API_KEY)
  return llm

model = load_model()

In [33]:
response = model.invoke(prompt)

In [34]:
print(response.content)

Based on the provided full plots, none of these films are explicitly described as horror movies.

*   **Godzilla vs. Biollante** is a giant monster (kaiju) film, which can sometimes have elements of horror due to the destructive nature of the monsters, but its primary genre is typically considered sci-fi or action.
*   **Search and Destroy** appears to be a crime or dark comedy film, with a mention of a horror story being written by a character, but not that the movie itself is horror.
*   **Megiddo: The Omega Code 2** is described as a supernatural film with apocalyptic themes, which can overlap with horror, but its focus on a Caine and Abel dynamic and the battle for the world's fate suggests it might lean more towards action or religious thriller genres.

Therefore, based solely on these descriptions, it's not possible to identify the "best horror movie" from this selection.
